In [1]:
#| default_exp updates
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path

# Insert in Path Project Directory
sys.path.insert(0, str(Path().cwd().parent))

# Atualização

> Este módulo atualiza as bases. Executa as queries sql do STEL, RADCOM e baixa os arquivos de estações e plano básico do MOSAICO.

In [2]:
#| export
from decimal import Decimal, getcontext
from typing import Union
from urllib.request import urlretrieve, URLError
import xmltodict
from zipfile import ZipFile
import gc

import pandas as pd
import pyodbc
from rich.console import Console
from pyarrow import ArrowInvalid, ArrowTypeError
from unidecode import unidecode
from fastcore.xtras import Path
from fastcore.foundation import L
from fastcore.test import test_eq
from tqdm.auto import tqdm
import pyodbc
from pymongo import MongoClient

from anateldb.constants import *
from anateldb.format import parse_bw, format_types, input_coordenates
from anateldb.functionsdb import ConsultaSRD

getcontext().prec = 5

## Conexão com o banco de dados
A função a seguir é um `wrapper` simples que utiliza o `pyodbc` para se conectar ao banco de dados base da Anatel e retorna o objeto da conexão

In [3]:
#| export
def connect_db(server: str = 'ANATELBDRO05', # Servidor do Banco de Dados
               database: str = 'SITARWEB', # Nome do Banco de Dados
               trusted_conn: str = 'yes', # Conexão Segura: yes | no
               mult_results: bool = True, # Múltiplos Resultados
              )->pyodbc.Connection:
    """Conecta ao Banco `server` e retorna o 'cursor' (iterador) do Banco"""
    return pyodbc.connect(
        "Driver={ODBC Driver 17 for SQL Server};"
        f"Server={server};"
        f"Database={database};"
        f"Trusted_Connection={trusted_conn};"
        f"MultipleActiveResultSets={mult_results};",
        timeout=TIMEOUT,
    )

In [4]:
#echo: false
def test_connection():
    conn = connect_db()
    cursor = conn.cursor()
    for query in (RADCOM,STEL):
        cursor.execute(query)
        test_eq(type(cursor.fetchone()), pyodbc.Row)

In [5]:
#| exporti
def _parse_estações(row: dict)->dict:
    """Given a row in a MongoDB ( a dict of dicts ), it travels some keys and return a subset dict"""
    
    d = {k.replace('@', '').lower():row[k] for k in ("@SiglaServico", "@id", "@state",
        "@entidade",
        "@fistel",
        "@cnpj",
        "@municipio",
        "@uf")}
    entidade = row.get('entidade', {})
    d.update({k.replace('@', '').lower():entidade[k] for k in ('@num_servico', '@habilitacao_DataValFreq')})
    administrativo = row.get('administrativo', {})
    d['numero_estacao'] = administrativo.get('@numero_estacao')
    estacao = row.get('estacao_principal', {})
    d.update({k.replace('@', '').lower():estacao[k] for k in ('@latitude', '@longitude')})
    return d

In [6]:
#| exporti
def _read_estações(path: Union[str, Path]) -> pd.DataFrame:
    """Read the zipped xml file `Estações.zip` from MOSAICO and returns a dataframe"""
    
    with ZipFile(path) as myzip:
        with myzip.open('estacao_rd.xml') as myfile:
            estacoes = xmltodict.parse(myfile.read())
            
    assert 'estacao_rd' in estacoes, "The xml file inside estacoes.zip is not in the expected format"
    assert 'row' in estacoes['estacao_rd'], "The xml file inside estacoes.zip is not in the expected format"
    
    df = pd.DataFrame(L(estacoes['estacao_rd']['row']).map(_parse_estações))
    df = df[df.state.str.contains("-C1$|-C2$|-C3$|-C4$|-C7|-C98$")].reset_index(drop=True)
    df = df.loc[:, COL_ESTACOES]
    df.columns = NEW_ESTACOES    
    for c in df.columns:
        df.loc[df[c] == "", c] = pd.NA
    return df

In [7]:
#| exporti
def _parse_pb(row: dict)->dict:
    """Given a row in the MongoDB file canais.zip ( a dict of dicts ), it travels some keys and return a subset dict"""
    return {unidecode(k).lower().replace("@", ""): v  for k,v in row.items()}

In [8]:
#| exporti
def _read_plano_basico(path: Union[str, Path]) -> pd.DataFrame:
    """Read the zipped xml file `Plano_Básico.zip` from MOSAICO and returns a dataframe"""    
    df = L()
    with ZipFile(path) as myzip:
        with myzip.open('plano_basicoTVFM.xml') as myfile:
            pbtvfm = xmltodict.parse(myfile.read())
        with myzip.open('plano_basicoAM.xml') as myfile:
            pbam = xmltodict.parse(myfile.read())
        with myzip.open('secundariosTVFM.xml') as myfile:
            stvfm = xmltodict.parse(myfile.read())
        with myzip.open('secundariosAM.xml') as myfile:
            sam = xmltodict.parse(myfile.read())    
            
    for base in (pbtvfm, stvfm, pbam, sam):
        assert 'plano_basico' in base, "The xml files inside canais.zip is not in the expected format"
        assert 'row' in base['plano_basico'], "The xml file inside canais.zip is not in the expected format"
        df.extend(L(base['plano_basico']['row']).map(_parse_pb))
        
    df = pd.DataFrame(df)
    df = df.loc[df.pais == "BRA", COL_PB].reset_index(drop=True)    
    df.columns = NEW_PB
    df = df[df.Status.str.contains("-C1$|-C2$|-C3$|-C4$|-C7|-C98$")].reset_index(drop=True)
    df.loc[:, 'Frequência'] = df.Frequência.str.replace(',', '.')
    for c in df.columns:
        df.loc[df[c] == '', c] = pd.NA
    return df    

In [9]:
#| export
def clean_mosaico(df: pd.DataFrame, # DataFrame com os dados de Estações e Plano_Básico mesclados 
                pasta: Union[str, Path], # Pasta com os dados de municípios para imputar coordenadas ausentes
) -> pd.DataFrame: # DataFrame com os dados mesclados e limpos
    """Clean the merged dataframe with the data from the MOSAICO page"""
    COLS = [c for c in df.columns if "_x" in c]
    for col in COLS:
        col_x = col
        col_y = col.split("_")[0] + "_y"
        df.loc[df[col_x].isna(), col_x] = df.loc[df[col_x].isna(), col_y]
        df.loc[df[col_y].isna(), col_y] = df.loc[df[col_y].isna(), col_x]
        if df[col_x].notna().sum() > df[col_y].notna().sum():
            a, b = col_x, col_y
        else:
            a, b = col_y, col_x
        df.drop(b, axis=1, inplace=True)
        df.rename({a: a[:-2]}, axis=1, inplace=True)

    # df.loc[df.Latitude_Transmissor == "", "Latitude_Transmissor"] = df.loc[
    #     df.Latitude_Transmissor == "", "Latitude_Estação"
    # ]
    # df.loc[df.Longitude_Transmissor == "", "Longitude_Transmissor"] = df.loc[
    #     df.Longitude_Transmissor == "", "Longitude_Estação"
    # ]
    
    # df.loc[df.Latitude_Transmissor.isna(), "Latitude_Transmissor"] = df.loc[
    #     df.Latitude_Transmissor.isna(), "Latitude_Estação"
    # ]
    # df.loc[df.Longitude_Transmissor.isna(), "Longitude_Transmissor"] = df.loc[
    #     df.Longitude_Transmissor.isna(), "Longitude_Estação"
    # ]
    # df.drop(["Latitude_Estação", "Longitude_Estação"], axis=1, inplace=True)
    # df.rename(
    #     columns={
    #         "Latitude_Transmissor": "Latitude",
    #         "Longitude_Transmissor": "Longitude",
    #     },
    #     inplace=True,
    # )

    df = input_coordenates(df, pasta)
    df.loc[:, "Frequência"] = df.Frequência.str.replace(",", ".")    
    df = df[df.Frequência.notna()].reset_index(drop=True)

    # Removido o código abaixo devido a inconsistência no Mosaico
    # if freq_nans := df[df.Frequência.isna()].Id.tolist():
    #     complement_df = scrape_dataframe(freq_nans)
    #     df.loc[
    #         df.Frequência.isna(),
    #         [
    #             "Status",
    #             "Entidade",
    #             "Fistel",
    #             "Frequência",
    #             "Classe",
    #             "Num_Serviço",
    #             "Município",
    #             "UF",
    #         ],
    #     ] = complement_df.values

    df.loc[:, "Frequência"] = df.Frequência.astype("float")
    df.loc[df.Serviço == "OM", "Frequência"] = df.loc[
        df.Serviço == "OM", "Frequência"
    ].apply(lambda x: Decimal(x) / Decimal(1000))
    df.loc[:, "Validade_RF"] = df.Validade_RF.astype("string").str.slice(0, 10)
    return df.drop_duplicates(keep="first").reset_index(drop=True)

## Atualização das bases de dados
As bases de dados são atualizadas atráves das funções a seguir, o único argumento passado em todas elas é a pasta na qual os arquivos locais processados serão salvos, os nomes dos arquivos são padronizados e não podem ser editados para que as funções de leitura e processamento recebam somente a pasta na qual esses arquivos foram salvos.

In [10]:
#| export
def _save_df(df: pd.DataFrame, folder: Union[str, Path], stem: str) -> pd.DataFrame:
    """Format, Save and return a dataframe"""
    df = format_types(df, stem)
    df = df.drop_duplicates(keep='first').reset_index(drop=True)
    df = df.dropna(subset=['Latitude', 'Longitude']).reset_index(drop=True)
    try:
        file = Path(f"{folder}/{stem}.parquet.gzip")
        df.to_parquet(file, compression="gzip", index=False)
    except (ArrowInvalid, ArrowTypeError):
        file.unlink(missing_ok=True)
        try:
            file = Path(f"{folder}/{stem}.fth")
            df.to_feather(file)
        except (ArrowInvalid, ArrowTypeError):
            file.unlink(missing_ok=True)
            try:
                file = Path(f"{folder}/{stem}.xlsx")
                with pd.ExcelWriter(file) as wb:
                    df.to_excel(
                        wb, sheet_name="DataBase", engine="openpyxl", index=False
                    )
            except Exception as e:
                raise ValueError(f"Could not save {stem} to {file}") from e
    return df



In [11]:
# | export
def update_radcom(
        conn: pyodbc.Connection, # Objeto de conexão de banco
        folder: Union[str, Path] # Pasta onde salvar os arquivos
        
) -> pd.DataFrame: # DataFrame com os dados atualizados
    """Atualiza a tabela local retornada pela query `RADCOM`"""
    console = Console()
    with console.status(
        "[cyan]Lendo o Banco de Dados de Radcom...", spinner="earth"
    ) as status:
        try:            
            df = pd.read_sql_query(RADCOM, conn)
            return _save_df(df, folder, "radcom")
        except pyodbc.OperationalError as e:
            status.console.log(
                "Não foi possível abrir uma conexão com o SQL Server. Esta conexão somente funciona da rede cabeada!"
            )
            raise ConnectionError from e

In [12]:
#|export
def update_stel(
        conn: pyodbc.Connection, # Objeto de conexão de banco
        folder:Union[str, Path] # Pasta onde salvar os arquivos        
) -> pd.DataFrame: # DataFrame com os dados atualizados
    """Atualiza a tabela local retornada pela query `STEL`"""
    console = Console()
    with console.status(
        "[red]Lendo o Banco de Dados do STEL. Processo Lento, aguarde...",
        spinner="bouncingBall",
    ) as status:
        try:            
            df = pd.read_sql_query(STEL, conn)
            return _save_df(df, folder, "stel")
        except pyodbc.OperationalError as e:
            status.console.log(
                "Não foi possível abrir uma conexão com o SQL Server. Esta conexão somente funciona da rede cabeada!"
            )
            raise ConnectionError from e

In [13]:
# stel = update_stel(Path.cwd().parent / 'dados')

In [14]:
#|export
def update_mosaico(        
        mongo_client: MongoClient, # Objeto de conexão com o MongoDB
        folder: Union[str, Path] # Pasta onde salvar os arquivos
) -> pd.DataFrame: # DataFrame com os dados atualizados
    """Efetua a query na tabela de Radiodifusão no banco mongoDB `mongo_client` e atualiza o arquivo local"""
    console = Console()
    with console.status(
        "Consolidando os dados do Mosaico...", spinner="clock"
    ) as status:  
        
        database = mongo_client["sms"]
        # Database com as informações de Radio e difusão
        collection = database["srd"]

        query = {}
        projection = {"SiglaServico": 1.0, "Status.state": 1.0, "licensee": 1.0, "NumFistel": 1.0, "frequency": 1.0, "stnClass": 1.0, "srd_planobasico.NomeMunicipio": 1.0, "srd_planobasico.SiglaUF": 1.0, "NumServico": 1.0, "estacao.NumEstacao": 1.0, "estacao.MedLatitudeDecimal": 1.0, "estacao.MedLongitudeDecimal": 1.0, "habilitacao.DataValFreq": 1.0}

        list_data = list(collection.find(query, projection = projection))
        mosaico_df = pd.json_normalize(list_data)
        mosaico_df = mosaico_df.drop(columns=['estacao'])
        mosaico_df = mosaico_df[["frequency",
                                "licensee",
                                "NumFistel",
                                "estacao.NumEstacao",
                                "srd_planobasico.NomeMunicipio",
                                "srd_planobasico.SiglaUF",
                                "estacao.MedLatitudeDecimal",
                                "estacao.MedLongitudeDecimal",
                                "stnClass",
                                "NumServico",
                                "habilitacao.DataValFreq",
                                "Status.state"]]

        mosaico_df.columns = RADIODIFUSAO
        mosaico_df = mosaico_df[mosaico_df.Status.str.contains("-C1$|-C2$|-C3$|-C4$|-C7|-C98$", na=False)].reset_index(drop=True)
        for c in mosaico_df.columns:
            mosaico_df.loc[mosaico_df[c] == "", c] = pd.NA
        mosaico_df = mosaico_df.dropna(subset=['UF'])
        mosaico_df = mosaico_df[mosaico_df.Frequência.notna()].reset_index(drop=True)

        df = clean_mosaico(mosaico_df, folder)
    return _save_df(df, folder, "mosaico")

In [15]:
#| export
def update_licenciamento(mongo_client: MongoClient, # Objeto de conexão com o MongoDB
                         folder: Union[str, Path] # Pasta onde salvar os arquivos
)-> pd.DataFrame: # DataFrame com os dados atualizados
    """Efetua a query na tabela `licenciamento` no banco mongoDB `mongo_client` e atualiza o arquivo local"""
    database = mongo_client["sms"]
    collection = database["licenciamento"]    
    c = collection.find(MONGO_LIC, projection={k:1.0 for k in LICENCIAMENTO.keys()})
    result = L()
    for doc in tqdm(c):
        result.append(doc)
    df = pd.json_normalize(result)
    df.drop('_id', axis=1, inplace=True)
    df.rename(LICENCIAMENTO, axis=1, inplace=True)
    df['Designacao_Emissão'] = df.Designacao_Emissão.str.replace(',', ' ')
    df['Designacao_Emissão'] = df.Designacao_Emissão.str.strip().str.lstrip().str.rstrip().str.upper()
    df['Designacao_Emissão'] = df.Designacao_Emissão.str.split(' ')
    df = df.explode('Designacao_Emissão')
    df.loc[df.Designacao_Emissão == '/', 'Designacao_Emissão'] = ''
    df.loc[:, ['Largura_Emissão', 'Classe_Emissão']]  = df.Designacao_Emissão.apply(parse_bw).tolist()
    df.drop('Designacao_Emissão', axis=1, inplace=True)
    subset = ['Entidade', 'Longitude', 'Latitude', 'Classe', 'Frequência', 'Num_Serviço', 'Largura_Emissão', 'Classe_Emissão']
    df_sub = df[~df.duplicated(subset=subset, keep='first')].reset_index(drop=True).copy()
    df_sub = df_sub.set_index(subset).sort_index()
    df_sub['Count'] = (df.groupby(subset).count()['Número_Estação']).tolist()
    del df ; gc.collect()
    df_sub = df_sub.reset_index()
    return _save_df(df_sub, folder, 'licenciamento')

In [17]:
#|hide
uri = input()
mongo_client = MongoClient(uri)
mongo_client.server_info()

{'version': '4.0.5',
 'gitVersion': '3739429dd92b92d1b0ab120911a23d50bf03c412',
 'targetMinOS': 'Windows 7/Windows Server 2008 R2',
 'modules': [],
 'allocator': 'tcmalloc',
 'javascriptEngine': 'mozjs',
 'sysInfo': 'deprecated',
 'versionArray': [4, 0, 5, 0],
 'openssl': {'running': 'Windows SChannel'},
 'buildEnvironment': {'distmod': '2008plus-ssl',
  'distarch': 'x86_64',
  'cc': 'cl: Microsoft (R) C/C++ Optimizing Compiler Version 19.00.24223 for x64',
  'ccflags': '/nologo /EHsc /W3 /wd4355 /wd4800 /wd4267 /wd4244 /wd4290 /wd4068 /wd4351 /wd4373 /we4013 /we4099 /we4930 /WX /errorReport:none /MD /O2 /Oy- /bigobj /utf-8 /Zc:rvalueCast /Zc:strictStrings /volatile:iso /Gw /Gy /Zc:inline',
  'cxx': 'cl: Microsoft (R) C/C++ Optimizing Compiler Version 19.00.24223 for x64',
  'cxxflags': '/TP',
  'linkflags': '/nologo /DEBUG /INCREMENTAL:NO /LARGEADDRESSAWARE /OPT:REF',
  'target_arch': 'x86_64',
  'target_os': 'windows'},
 'bits': 64,
 'debug': False,
 'maxBsonObjectSize': 16777216,
 '

In [18]:
import warnings
warnings.filterwarnings("ignore", message='install "ipywidgets" for Jupyter support')

folder = Path.cwd().parent / 'dados'

In [19]:
df = update_licenciamento(mongo_client, folder)

0it [00:00, ?it/s]

In [20]:
df

,Entidade,Longitude,Latitude,Classe,Frequência,Num_Serviço,Largura_Emissão,Classe_Emissão,UF,Cod_Município,Município,Data_de_Validade,Número_Estação,Fistel,Status,Count
0,,-42.0827,-4.02389,FX,7807.0,019,28000.0,D7W,PI,2201507,BATALHA,2039-11-28,1012185475,50418766738,LIC-LIC-01,1
1,ALVES & BATISTA ALARMES LTDA - ME,-48.984178,-23.115239,FX,149.19,019,11.0,F3E,SP,3504503,,2038-05-24,1006566675,50415983045,LIC-LIC-01,1
2,ALVES & BATISTA ALARMES LTDA - ME,-48.984178,-23.115239,FX,149.19,019,16.0,F3E,SP,3504503,,2038-05-24,1006566675,50415983045,LIC-LIC-01,1
3,ALVES & BATISTA ALARMES LTDA - ME,-48.928081,-23.098297,FX,149.19,019,11.0,F3E,SP,3504503,,2038-05-24,1006566640,50415983045,LIC-LIC-01,1
4,ALVES & BATISTA ALARMES LTDA - ME,-48.928081,-23.098297,FX,149.19,019,16.0,F3E,SP,3504503,,2038-05-24,1006566640,50415983045,LIC-LIC-01,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
735507,pedro luis lorenzetti,-48.836389,-22.631667,BR,171.06875,019,11.0,F3E,SP,3526803,,2030-09-05,534834183,50009553231,LIC-LIC-01,1
735508,pedro luis lorenzetti,-48.836389,-22.631667,ML,166.46875,019,11.0,F3E,SP,3526803,,2030-09-05 00:00:00.000,534834205,50009553231,LIC-LIC-01,92
735509,willian diogo da costa,-53.486667,-24.948889,ML,157.58125,019,7.6,F1W,PR,4104808,,2033-12-11,699459265,50410908827,LIC-LIC-01,12
735510,ÔMEGA SEGURANÇA E VIGILÂNCIA PATRIMONIAL LTDA ME,-35.734883,-9.627794,FB,161.37,019,7.6,F1D,AL,2704302,,2033-10-10,699260540,50410720518,LIC-LIC-01,1


In [33]:
# | export
def update_base(
    conn: pyodbc.Connection, # Objeto de conexão de banco
    clientMongoDB: MongoClient, # Ojeto de conexão com o MongoDB
    folder: Union[str, Path] # Pasta onde salvar os arquivos    
) -> pd.DataFrame: # DataFrame com os dados atualizados
    # sourcery skip: use-fstring-for-concatenation
    """Wrapper que atualiza opcionalmente lê e atualiza as três bases indicadas anteriormente, as combina e salva o arquivo consolidado na folder `folder`"""
    stel = update_stel(conn, folder, ).loc[:, TELECOM]
    radcom = update_radcom(conn, folder).loc[:, SRD]
    mosaico = update_mosaico(clientMongoDB, folder).loc[:, RADIODIFUSAO]    
    radcom["Num_Serviço"] = "231"
    radcom["Status"] = "RADCOM"
    radcom["Classe_Emissão"] = pd.NA
    radcom["Largura_Emissão"] = BW_MAP["231"]
    radcom["Entidade"] = radcom.Entidade.str.rstrip().str.lstrip()
    radcom["Validade_RF"] = pd.NA
    radcom["Fonte"] = "SRD"
    stel["Status"] = "L"
    stel["Entidade"] = stel.Entidade.str.rstrip().str.lstrip()
    stel["Fonte"] = "STEL"
    mosaico["Fonte"] = "MOS"
    mosaico["Classe_Emissão"] = pd.NA
    mosaico["Largura_Emissão"] = mosaico.Num_Serviço.map(BW_MAP)
    rd = (
        pd.concat([mosaico, radcom, stel])
        .sort_values(["Frequência", "Latitude", "Longitude"])
        .reset_index(drop=True)
    )
    rd = rd.drop_duplicates(keep="first").reset_index(drop=True)
    rd["BW(kHz)"] = rd.Largura_Emissão.astype('string').fillna('-1').apply(parse_bw)
    return _save_df(rd, folder, "base")

In [ ]:
# from urllib.request import Request, urlopen
# from urllib.error import URLError
# req = Request(ESTACAO)
# try:
#     response = urlopen(req)
# except URLError as e:
#     if hasattr(e, 'reason'):
#         print('We failed to reach a server.')
#         print('Reason: ', e.reason)
#     elif hasattr(e, 'code'):
#         print('The server couldn\'t fulfill the request.')
#         print('Error code: ', e.code)
# else:
#     Path.cwd().joinpath('estações.zip').write_bytes(response.read())